# 项目说明

从 `MEU_to_code/MEU_selected_with_relation_GT` 找到 `北京证券交易所上市公司持续监管指引第8号——股份减持和持股管理.xlsx`, 这是我们需要处理的 MEU Coding 任务.

将每一个 MEU 发送给 LLM 完成 coding 任务, 保存到 `MEU_to_code/MEU_code/raw_response/北京证券交易所上市公司持续监管指引第8号——股份减持和持股管理.xlsx`

之后再解析该回复, 保存到`MEU_to_code/MEU_code/北京证券交易所上市公司持续监管指引第8号——股份减持和持股管理.xlsx`

# MEU情况

In [1]:
1500/1000000 * 8 * 1000

12.0

In [2]:
import pandas as pd

# 读取Excel文件
file_path = "MEU_to_code/MEU_selected_with_relation_GT/北京证券交易所上市公司持续监管指引第8号——股份减持和持股管理.xlsx"
df = pd.read_excel(file_path)

# 统计各类别出现次数
type_counts = df['type'].value_counts()

# 计算比例
type_proportions = type_counts / len(df) * 100

# 打印结果
print("各类别统计结果:")
print(type_counts)
print("\n各类别比例(%):")
print(type_proportions)

各类别统计结果:
type
实际执行单元                    36
无法简单执行的兜底条款，倡议条款和模糊的条款    30
为简化模型暂不考虑的单元              26
在标准的数据格式下不需额外考虑           11
立法宗旨等法律基本信息                4
Name: count, dtype: int64

各类别比例(%):
type
实际执行单元                    33.644860
无法简单执行的兜底条款，倡议条款和模糊的条款    28.037383
为简化模型暂不考虑的单元              24.299065
在标准的数据格式下不需额外考虑           10.280374
立法宗旨等法律基本信息                3.738318
Name: count, dtype: float64


In [3]:
MEU_to_code_df = df[df['type']=="实际执行单元"]
MEU_to_code_df.columns

Index(['MEU_id', 'subject', 'condition', 'constraint', 'contextual_info',
       'relation', 'target', 'type', 'comments', 'note'],
      dtype='object')

# Prompt模板

## 普通任务 prompt

In [ ]:
prompt_meu_coding_bse_08 = """
# 身份
你是一个精通法律的程序员. 擅长编写代码以检验特定案例对于特定法律要求的合规性. 

## MEU概念简述
MEU (Minimum Executable Unit) 是法律条文拆解出的最小合规单元，包含：
- MEU_id: MEU的编号, 通常为"MEU_n_k", 其中n是其所属的法条的编号, k是其在法条内部的编号
- subject: 责任主体（如"控股股东") 
- condition: 触发条件（如"减持股份")  
- constraint: 约束内容（如"提前15日公告") 
- contextual_info: 补充说明（如价格计算方式）


# 任务
你现在面临一个编写代码以检验某法律事件的合规性的任务, 你需要编写一个函数, 这个函数的输入是一个日频的pandas dataframe, 每一行是在某个交易日内公司的情况和股东的情况, 这个函数会检验该dataframe的每一行在某一个给定的法律最小可执行单元上的合规性, 标注在特定的列上, 并返回标注后的dataframe. 


# 更多要求
 - 你可以进行任意长度的思考, 然后用<CODE></CODE>包裹你的代码. 
 - 你需要按照下面这个框架编写代码: 1. 标记valid的subject(meu_n_k_subject); 2. 标记valid的condition(meu_n_k_condition); 3. 标记valid的constrain(meu_n_k_constrain); 4. 在dataframe中记录subject, condition和constrain的valid情况; 5. 计算整体的违约情况并标记在dataframe上(meu_n_k_compliance). 

# example
下面是一个例子:

## 输入
法律的最小可执行单元编号: MEU_4_1
法律的最小可执行单元内容: {{"subject": "上市公司大股东 | 董监高", "condition": "计划通过本所集中竞价或大宗交易减持股份", "constraint": "应当及时通知公司，并在首次卖出的15个交易日前向本所报告并预先披露减持计划", "contextual_info": NaN}}
必要的额外信息: NaN

## 输出
<CODE>
import pandas a pd

def check_meu_4_1(df):
    '''
    代码的注释要清晰, 展示你分别验证subject, condition和constrain的过程.
    "subject": "上市公司大股东 | 董监高", 
    "condition": "计划通过本所集中竞价或大宗交易减持股份", 
    "constraint": "应当及时通知公司，并在首次卖出的15个交易日前向本所报告并预先披露减持计划", 
    "contextual_info": NaN
    '''

    df['meu_4_2_subject'] = False  # 初始化subject列
    df['meu_4_2_condition'] = False  # 初始化condition列
    df['meu_4_2_constrain'] = False  # 初始化constrain列
    
    # 标记valid的subject
    is_major_shareholder = (
        df['股东身份'].isin(['控股股东', '实际控制人', '持股5%以上股东']) |
        (df['持股比例'] >= 0.05)
    )
    is_director = df['股东身份'].isin(['董监高'])
    valid_subject = is_major_shareholder | is_director

    # 标记valid的condition
    valid_condition = df['减持方式'].isin(['竞价交易', '大宗交易'])

    # 标记valid的constrain
    df['交易日差'] = (df['计划开始日'] - df['计划披露日']).dt.days
    has_plan = df['存在减持计划']
    valid_constraint = df['交易日差'] >= 15 & df['存在减持计划']
    
    # 标记各条件满足情况
    df.loc[valid_subject, 'meu_4_1_subject'] = True
    df.loc[valid_condition, 'meu_4_1_condition'] = True
    df.loc[valid_constraint, 'meu_4_1_constraint'] = True
    
    # 判断违规情况：满足主体和条件但不满足约束
    is_violation = (
        valid_subject & 
        valid_condition & 
        (~valid_constraint)
    )
    
    # 标记违规行
    df.loc[is_violation, 'meu_4_1_compliance'] = False
    
    return df

</CODE>
    
# 下面轮到你来完成这个任务
法律的最小可执行单元编号: {meu_id}
法律的最小可执行单元内容: {meu_content}
必要的额外信息: {note}


# 附加信息
## 你可以操作的dataframe的columns: 
['日期', '日收益率', '收盘价', '前收盘价', '上市日期', '发行价格', '净资产增长率', '净利润增长率', '净资产',
'净利润', '总股本', '每股净资产', '每股净利润', '公告类型', '公告日期', '复权因子', '收盘价减每股净资产',
'收盘价减每股净利润', '公司涉嫌证券期货违法犯罪事件', '持股比例', '持股数量', '股东身份', '减持方式', '拟减持原因',
'存在减持计划', '计划减持比例', '计划价格下限', '计划价格上限', '当日减持比例', '累计减持比例', '计划披露日',
'计划开始日', '计划结束日', '离任日期', '公司简称', '股东', '股东涉嫌证券期货违法犯罪事件']

## 其中: 
- 所有日期相关的列都已经是pd.datetime格式, ['日期']列本身就是交易日, 不需要额外计算交易日, 但是自然日需要计算. 
- 所有价格的单位都是元而非万元, 千元或百万元; 所有的比例都是小数而非百分数
- 这是一个二维的面板数据, 根据['日期', '公司简称', '股东']可以定位到特定公司的特定股东在特定交易日的状态, 以及公司在该日的状态. 如果你要分公司和股东地考虑一些合规要求, 需要考虑按照 ['公司简称', '股东']分组并在组内完成. 
- 交易方式有['竞价交易', '大宗交易', '融券卖出'], 竞价交易包括集中竞价. 
- ['公司/股东涉嫌证券期货违法犯罪事件'] 有['被中国证监会及其派出机构立案调查', '中国证监会及其派出机构立案调查结束', '行政处罚决定作出', '被司法机关立案侦查', '司法机关立案侦查结束', '被本所公开谴责']
- ['股东身份'] 有 ['董监高', '离任董监高', '控股股东', '实际控制人', '持股5%以上股东', '普通股东'], 这些标签不会并列出现, 因此有时候董监高/离任董监高持股在5%以上但并没有标明'持股5%以上股东', 你需要从['持股比例']手动筛选. 
- ['存在减持计划']是一个布尔值，表示该股东当天是否有减持计划
- 假设公司上市发行价就是最早的非空的收盘价, 公司发行时的控股股东和实际控制人就是第一个非空的收盘价的同一天的控股股东和实际控制人. 
- ['持股比例'] 全部是公开上市前取得的股份. 不考虑承诺和限制出售的股份. 

"""

## 含有refer_to的任务

In [ ]:
prompt_meu_coding_bse_08_with_refer_to = """
# 身份
你是一个精通法律的程序员. 擅长编写代码以检验特定案例对于特定法律要求的合规性. 

## MEU概念简述
MEU (Minimum Executable Unit) 是法律条文拆解出的最小合规单元，包含：
- MEU_id: MEU的编号, 通常为"MEU_n_k", 其中n是其所属的法条的编号, k是其在法条内部的编号
- subject: 责任主体（如"控股股东") 
- condition: 触发条件（如"减持股份")  
- constraint: 约束内容（如"提前15日公告") 
- contextual_info: 补充说明（如价格计算方式）


# 任务
你现在面临一个编写代码以检验某法律事件的合规性的任务, 你需要编写一个函数, 这个函数的输入是一个日频的pandas dataframe, 每一行是在某个交易日内公司的情况和股东的情况, 这个函数会检验该dataframe的每一行在某一个给定的法律最小可执行单元上的合规性, 标注在特定的列上, 并返回标注后的dataframe. 

# 更多要求
 - 你可以进行任意长度的思考, 然后用<CODE></CODE>包裹你的代码. 
 - 你需要按照下面这个框架编写代码: 1. 标记valid的subject(meu_n_k_subject); 2. 标记valid的condition(meu_n_k_condition); 3. 标记valid的constrain(meu_n_k_constrain); 4. 在dataframe中记录subject, condition和constrain的valid情况; 5. 计算整体的违约情况并标记在dataframe上(meu_n_k_compliance). 

# example
下面是一个例子:

## 输入
法律的最小可执行单元编号: MEU_4_1
法律的最小可执行单元内容: {{"subject": "上市公司大股东 | 董监高", "condition": "计划通过本所集中竞价或大宗交易减持股份", "constraint": "应当及时通知公司，并在首次卖出的15个交易日前向本所报告并预先披露减持计划", "contextual_info": NaN}}
必要的额外信息: NaN

## 输出
<CODE>
import pandas a pd

def check_meu_4_1(df):
    '''
    代码的注释要清晰, 展示你分别验证subject, condition和constrain的过程.
    "subject": "上市公司大股东 | 董监高", 
    "condition": "计划通过本所集中竞价或大宗交易减持股份", 
    "constraint": "应当及时通知公司，并在首次卖出的15个交易日前向本所报告并预先披露减持计划", 
    "contextual_info": NaN
    '''

    df['meu_4_2_subject'] = False  # 初始化subject列
    df['meu_4_2_condition'] = False  # 初始化condition列
    df['meu_4_2_constrain'] = False  # 初始化constrain列
    
    # 标记valid的subject
    is_major_shareholder = (
        df['股东身份'].isin(['控股股东', '实际控制人', '持股5%以上股东']) |
        (df['持股比例'] >= 0.05)
    )
    is_director = df['股东身份'].isin(['董监高'])
    valid_subject = is_major_shareholder | is_director

    # 标记valid的condition
    valid_condition = df['减持方式'].isin(['竞价交易', '大宗交易'])

    # 标记valid的constrain
    df['交易日差'] = (df['计划开始日'] - df['计划披露日']).dt.days
    has_plan = df['存在减持计划']
    valid_constraint = df['交易日差'] >= 15 & df['存在减持计划']
    
    # 标记各条件满足情况
    df.loc[valid_subject, 'meu_4_1_subject'] = True
    df.loc[valid_condition, 'meu_4_1_condition'] = True
    df.loc[valid_constraint, 'meu_4_1_constraint'] = True
    
    # 判断违规情况：满足主体和条件但不满足约束
    is_violation = (
        valid_subject & 
        valid_condition & 
        (~valid_constraint)
    )
    
    # 标记违规行
    df.loc[is_violation, 'meu_4_1_compliance'] = False
    
    return df

</CODE>
    
# 下面轮到你来完成这个任务
法律的最小可执行单元编号: {meu_id}
法律的最小可执行单元内容: {meu_content}
必要的额外信息: {note}

你被分配的MEU coding任务存在refer_to关系, 你可以额外参考下面这些信息:
{refer_to}


# 附加信息
## 你可以操作的dataframe的columns: 
['日期', '日收益率', '收盘价', '前收盘价', '上市日期', '发行价格', '净资产增长率', '净利润增长率', '净资产',
'净利润', '总股本', '每股净资产', '每股净利润', '公告类型', '公告日期', '复权因子', '收盘价减每股净资产',
'收盘价减每股净利润', '公司涉嫌证券期货违法犯罪事件', '持股比例', '持股数量', '股东身份', '减持方式', '拟减持原因',
'存在减持计划', '计划减持比例', '计划价格下限', '计划价格上限', '当日减持比例', '累计减持比例', '计划披露日',
'计划开始日', '计划结束日', '离任日期', '公司简称', '股东', '股东涉嫌证券期货违法犯罪事件']

## 其中: 
- 所有日期相关的列都已经是pd.datetime格式, ['日期']列本身就是交易日, 不需要额外计算交易日, 但是自然日需要计算. 
- 所有价格的单位都是元而非万元, 千元或百万元; 所有的比例都是小数而非百分数
- 这是一个二维的面板数据, 根据['日期', '公司简称', '股东']可以定位到特定公司的特定股东在特定交易日的状态, 以及公司在该日的状态. 如果你要分公司和股东地考虑一些合规要求, 需要考虑按照 ['公司简称', '股东']分组并在组内完成. 
- 交易方式有['竞价交易', '大宗交易', '融券卖出'], 竞价交易包括集中竞价. 
- ['公司/股东涉嫌证券期货违法犯罪事件'] 有['被中国证监会及其派出机构立案调查', '中国证监会及其派出机构立案调查结束', '行政处罚决定作出', '被司法机关立案侦查', '司法机关立案侦查结束', '被本所公开谴责']
- ['股东身份'] 有 ['董监高', '离任董监高', '控股股东', '实际控制人', '持股5%以上股东', '普通股东'], 这些标签不会并列出现, 因此有时候董监高/离任董监高持股在5%以上但并没有标明'持股5%以上股东', 你需要从['持股比例']手动筛选. 
- ['存在减持计划']是一个布尔值，表示该股东当天是否有减持计划
- 假设公司上市发行价就是最早的非空的收盘价, 公司发行时的控股股东和实际控制人就是第一个非空的收盘价的同一天的控股股东和实际控制人. 
- ['持股比例'] 全部是公开上市前取得的股份. 不考虑承诺和限制出售的股份. 

"""

# 异步调用LLM

In [ ]:
import asyncio
import json
from tqdm.asyncio import tqdm_asyncio
import pandas as pd
from utils.call_gpt import call_gpt_async
import os

async def process_meu_async(
    df,
    output_filename,
    max_concurrency=5,
    prompt_template=prompt_meu_coding_bse_08,
    encoding='utf-8-sig'
):
    """
    异步处理MEU数据框的函数
    
    参数：
    df: 输入数据框
    output_filename: 输出文件名（自动添加路径）
    max_concurrency: 最大并行请求数 (默认5)
    encoding: 文件编码 (默认utf-8-sig)
    """
    # 状态管理类
    class ProcessingState:
        def __init__(self):
            self.total_prompt_tokens = 0
            self.total_completion_tokens = 0
            self.success_count = 0
            self.failure_count = 0
            self.results = []
            self.lock = asyncio.Lock()

    state = ProcessingState()

    async def process_row(row):
        """处理单行数据的异步任务"""
        nonlocal state
        try:
            # 构造meu_content JSON
            
            meu_content = json.dumps({
                "subject": row['subject'],
                "condition": row['condition'],
                "constraint": row['constraint'],
                "contextual_info": row['contextual_info']
            }, ensure_ascii=False)
            # print(meu_content)


            # 生成prompt
            prompt = prompt_template.format(
                meu_id=row['MEU_id'],
                meu_content=meu_content,
                note=row['note']
            )
            
            # print(prompt)

            # 调用异步接口
            content, reasoning_content, api_usage = await call_gpt_async(
                prompt=prompt,
                api_key="35684824-1776-48b6-94fd-96c2e99d0724",
                base_url="https://ark.cn-beijing.volces.com/api/v3",
                # model="ep-20250217153824-9xcbx", # deepseek-r1-250120
                model="ep-20250329012323-5qdxc", # deepseek-v3-250324
                # temperature=0.6,
            )

            # 更新共享状态
            async with state.lock:
                state.total_prompt_tokens += api_usage.prompt_tokens
                state.total_completion_tokens += api_usage.completion_tokens
                state.success_count += 1
                state.results.append({
                    **row.to_dict(),
                    'llm_response': content,
                    'reasoning_content': reasoning_content,
                    'prompt_tokens': api_usage.prompt_tokens,
                    'completion_tokens': api_usage.completion_tokens
                })

            return True
        except Exception as e:
            async with state.lock:
                state.failure_count += 1
                print(f"处理MEU {row['MEU_id']}失败: {str(e)}")
            return False

    # 创建异步任务
    tasks = [process_row(row) for _, row in df.iterrows()]
    
    # 配置输出路径
    output_dir = "MEU_to_code/MEU_code/raw_response"
    os.makedirs(output_dir, exist_ok=True)
    output_path = os.path.join(output_dir, output_filename)

    # 使用进度条处理
    with tqdm_asyncio(total=len(tasks), desc="处理MEU条目") as pbar:
        for i in range(0, len(tasks), max_concurrency):
            batch = tasks[i:i+max_concurrency]
            await asyncio.gather(*batch)
            pbar.update(len(batch))

    # 保存结果到Excel
    result_df = pd.DataFrame(state.results)
    
    # 添加统计信息到文件末尾
    stats_df = pd.DataFrame([{
        "total_prompt_tokens": state.total_prompt_tokens,
        "total_completion_tokens": state.total_completion_tokens,
        "success_count": state.success_count,
        "failure_count": state.failure_count
    }])
    
    with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
        result_df.to_excel(writer, index=False)
        # stats_df.to_excel(writer, sheet_name='统计信息', index=False)
    
    csv_path = output_path.replace('.xlsx', '.csv')
    result_df.to_csv(csv_path, index=False)

    print(f"处理完成！结果已保存至：{output_path}")



In [ ]:
# 在Jupyter中使用时这样调用
await process_meu_async(
    MEU_to_code_df,
    "北京证券交易所上市公司持续监管指引第8号——股份减持和持股管理.xlsx"
)


# 解析出代码并保存

In [ ]:
import pandas as pd
import re

# 读取原始Excel文件
input_path = "MEU_to_code/MEU_code/raw_response/北京证券交易所上市公司持续监管指引第8号——股份减持和持股管理.xlsx"
df = pd.read_excel(input_path)

# 定义一个函数来提取<CODE>标签中的内容
def extract_code(text):
    if not isinstance(text, str):
        return ""
    
    # 处理<\CODE>的错误格式，统一替换为</CODE>
    text = text.replace('<\\CODE>', '</CODE>')
    
    # 使用正则表达式提取所有<CODE>...</CODE>之间的内容
    code_blocks = re.findall(r'<CODE>(.*?)</CODE>', text, re.DOTALL)
    
    # 将所有代码块合并，用两个换行符分隔
    return '\n\n'.join(code_blocks).strip()

# 应用函数提取代码
df['code'] = df['llm_response'].apply(extract_code)

# 保存到新的Excel文件
output_path = "MEU_to_code/MEU_code/北京证券交易所上市公司持续监管指引第8号——股份减持和持股管理.xlsx"
df.to_excel(output_path, index=False)

print(f"处理完成，结果已保存到: {output_path}")

# 保存到ipynb方便查看

In [ ]:
import pandas as pd
import nbformat
from nbformat.v4 import new_notebook, new_markdown_cell, new_code_cell
import re

# 读取Excel文件
input_path = "MEU_to_code/MEU_code/北京证券交易所上市公司持续监管指引第8号——股份减持和持股管理.xlsx"
df = pd.read_excel(input_path)

# 创建新的Notebook
notebook = new_notebook()

# 提取法条号(n)和MEU序号(k)用于排序
def extract_n(x):
    match = re.match(r'MEU_(\d+)_\d+', str(x))
    return int(match.group(1)) if match else 0

def extract_k(x):
    match = re.match(r'MEU_\d+_(\d+)', str(x))
    return int(match.group(1)) if match else 0

df['MEU_n'] = df['MEU_id'].apply(extract_n)
df['MEU_k'] = df['MEU_id'].apply(extract_k)

# 按法条号(n)和MEU序号(k)排序
df = df.sort_values(['MEU_n', 'MEU_k'])

# 遍历每个MEU（已排序）
current_n = None
for _, row in df.iterrows():
    # 如果法条号变化，添加一级标题
    if row['MEU_n'] != current_n:
        notebook.cells.append(new_markdown_cell(f"# Law Article {row['MEU_n']}"))
        current_n = row['MEU_n']
    
    # 添加二级标题（MEU编号）
    notebook.cells.append(new_markdown_cell(f"## {row['MEU_id']}"))
    
    # 创建MEU信息表格
    info_table = f"""
| 字段 | 内容 |
|------|------|
| subject | {row['subject'] or ''} |
| condition | {row['condition'] or ''} |
| constraint | {row['constraint'] or ''} |
| contextual_info | {row['contextual_info'] or ''} |
| note | {row['note'] or ''} |
| relation | {row['relation'] or ''} |
| target | {row['target'] or ''} |
| type | {row['type'] or ''} |
| comments | {row['comments'] or ''} |
| prompt_tokens | {row['prompt_tokens'] or ''} |
| completion_tokens | {row['completion_tokens'] or ''} |
"""
    notebook.cells.append(new_markdown_cell(info_table))
    
    # 添加reasoning_content
    if pd.notna(row['reasoning_content']):
        notebook.cells.append(new_markdown_cell("### 推理内容"))
        notebook.cells.append(new_markdown_cell(str(row['reasoning_content'])))
    
    # 添加代码单元格
    if pd.notna(row['code']):
        notebook.cells.append(new_markdown_cell("### 代码实现"))
        notebook.cells.append(new_code_cell(str(row['code'])))
    
    # 添加分隔线
    notebook.cells.append(new_markdown_cell("---"))

# 保存Notebook文件
output_path = "MEU_to_code/MEU_code/view_ipynb/北京证券交易所上市公司持续监管指引第8号——股份减持和持股管理.ipynb"
with open(output_path, 'w', encoding='utf-8') as f:
    nbformat.write(notebook, f)

print(f"Jupyter Notebook已成功生成: {output_path}")